In [6]:
%matplotlib inline
import os
import importlib
import yaml

import xarray as xr
import numpy as np

import intake
import intake_cesm
import xcollection as xc

DEBUG:matplotlib.pyplot:Loaded backend module://ipykernel.pylab.backend_inline version unknown.


In [7]:
col = intake.open_cesm_metadatastore('cesm1_le')
col.df.info()

Active collection: cesm1_le
<class 'pandas.core.frame.DataFrame'>
Int64Index: 262092 entries, 0 to 262091
Data columns (total 18 columns):
resource            262092 non-null object
resource_type       262092 non-null object
direct_access       262092 non-null bool
experiment          262092 non-null object
case                262092 non-null object
component           262092 non-null object
stream              262092 non-null object
variable            262092 non-null object
date_range          262092 non-null object
ensemble            262092 non-null int64
files               262092 non-null object
files_basename      262092 non-null object
files_dirname       262092 non-null object
ctrl_branch_year    0 non-null float64
year_offset         34112 non-null float64
sequence_order      262092 non-null int64
has_ocean_bgc       262092 non-null bool
grid                52764 non-null object
dtypes: bool(2), float64(2), int64(2), object(12)
memory usage: 34.5+ MB


In [8]:
query = {'experiment': ['20C', 'RCP85'],
         'ensemble': 2, 
         'variable': 'TEMP',
         'direct_access': True}

cat = col.search(**query)
cat.results

,resource,resource_type,direct_access,experiment,case,component,stream,variable,date_range,ensemble,files,files_basename,files_dirname,ctrl_branch_year,year_offset,sequence_order,has_ocean_bgc,grid
225049,GLADE:posix:/glade/collections/cdg/data/cesmLE...,posix,True,20C,b.e11.B20TRC5CNBDRD.f09_g16.002,ocn,pop.h,TEMP,192001-200512,2,/glade/collections/cdg/data/cesmLE/CESM-CAM5-B...,b.e11.B20TRC5CNBDRD.f09_g16.002.pop.h.TEMP.192...,/glade/collections/cdg/data/cesmLE/CESM-CAM5-B...,NaN,NaN,0,True,POP_gx1v6
140428,GLADE:posix:/glade/collections/cdg/data/cesmLE...,posix,True,RCP85,b.e11.BRCP85C5CNBDRD.f09_g16.002,ocn,pop.h,TEMP,200601-208012,2,/glade/collections/cdg/data/cesmLE/CESM-CAM5-B...,b.e11.BRCP85C5CNBDRD.f09_g16.002.pop.h.TEMP.20...,/glade/collections/cdg/data/cesmLE/CESM-CAM5-B...,NaN,NaN,1,True,POP_gx1v6
140429,GLADE:posix:/glade/collections/cdg/data/cesmLE...,posix,True,RCP85,b.e11.BRCP85C5CNBDRD.f09_g16.002,ocn,pop.h,TEMP,208101-210012,2,/glade/collections/cdg/data/cesmLE/CESM-CAM5-B...,b.e11.BRCP85C5CNBDRD.f09_g16.002.pop.h.TEMP.20...,/glade/collections/cdg/data/cesmLE/CESM-CAM5-B...,NaN,NaN,1,True,POP_gx1v6


In [9]:
recipes = 'analysis_recipes.yml'
with open('analysis_recipes.yml') as fid:
    recipes = yaml.load(fid)
print(yaml.dump(recipes))

annmean:
  description: compute annual means
  file_format: nc
  operators:
  - !operator
    applied_method: time:mean
    function: compute_ann_mean
    kwargs: {}
    module: esmlab.climatology
monanom:
  description: compute monthly anomalies
  file_format: nc
  operators:
  - !operator
    applied_method: time:anom_mon
    function: compute_mon_anomaly
    kwargs: {}
    module: esmlab.climatology
monclim:
  description: compute monthly climatology
  file_format: nc
  name: monclim
  operators:
  - !operator
    applied_method: time:clim_mon
    function: compute_mon_climatology
    kwargs: {}
    module: esmlab.climatology
monclim_1990s:
  description: compute monthly climatology over 1990-1999
  file_format: nc
  name: monclim_1990s
  operators:
  - !operator
    applied_method: time:clim_mon
    function: compute_mon_climatology
    kwargs: {}
    module: esmlab.climatology
  sel_kwargs:
    time: !!python/object/apply:builtins.slice ['1990-01-01', '1999-12-31', null]



In [10]:
dc = xc.analyzed_collection(collection='cesm1_le',
                            analysis_recipe=recipes['monclim_1990s'],
                            overwrite_existing=True,
                            file_format="nc",
                            **query)
dc

Active collection: cesm1_le


INFO:root:applying sel_kwargs: {'time': slice('1990-01-01', '1999-12-31', None)}
INFO:root:applying operator: <function compute_mon_climatology at 0x2b7c00e910d0>
INFO:root:writing /glade/scratch/mclong/xcollection-data/cesm1_le-a3c1d024-c8b8-437b-834c-541fa29f3462-monclim_1990s.002.nc
/glade/work/mclong/miniconda3/envs/arctic_project/lib/python3.6/site-packages/dask/array/numpy_compat.py:28: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


In [11]:
%load_ext watermark

In [12]:
%watermark --iversion -g -h -m -v -u -d

intake      0.4.1
xcollection v0.10+8.g59df6fa
numpy       1.16.1
xarray      0.11.3
yaml        3.13
intake_cesm v2019.2.0+45.ge2de240
last updated: 2019-02-06 

CPython 3.6.7
IPython 7.1.1

compiler   : GCC 7.3.0
system     : Linux
release    : 3.10.0-693.21.1.el7.x86_64
machine    : x86_64
processor  : x86_64
CPU cores  : 72
interpreter: 64bit
host name  : casper05
Git hash   : 04a19cd50e1443dc253547f44c41f3a666dde712
